In [1]:
import subprocess
filename = "./maple10busnodebreaker.xml"
url = "http://localhost:8889/bigdata/namespace/kb/sparql"
subprocess.call(["curl", "-s", "-D-", "-H", "Content-Type: application/xml", "--upload-file", filename, "-X", "Post", url])


HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2615" milliseconds="22"/>

0

In [2]:
import importlib
cim_profile = "rc4_2021"
cim = importlib.import_module("cimgraph.data_profile." + cim_profile)

In [3]:
from cimgraph.databases import Parameter, ConnectionParameters
from cimgraph.databases.blazegraph.blazegraph import BlazegraphConnection
# from cimgraph.databases.gridappsd import GridappsdConnection, get_topology_response
from cimgraph.models import NodeBreakerModel
import json
import time

In [4]:
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql",  cim_profile=cim_profile)
bg = BlazegraphConnection(params)

In [5]:
geo_region_id = "_EE4C60AE-550D-4599-92F4-022DF3118B3C"
geo_region = cim.GeographicalRegion(mRID = geo_region_id)

In [6]:
network = NodeBreakerModel(connection=bg, container=geo_region, distributed=False)
network.add_to_graph(geo_region)

In [7]:
cim_class = cim.SubGeographicalRegion

In [8]:
network.get_all_edges(cim_class)
network.pprint(cim_class)

{}


In [9]:
feeder = {}
feeder["container"] = "Feeder"
feeder["contains"] = []

bay = {}
bay["container"] = "Bay"
bay["contains"] = []

voltage_level = {}
voltage_level["container"] = "VoltageLevel"
voltage_level["contains"] = [bay]

sub_area = {}
sub_area["container"] = "Substation"
sub_area["contains"] = [voltage_level, feeder]

subregion = {}
subregion["container"] = "SubGeographicalRegion"
subregion["contains"] = [sub_area]

region = {}
region["container"] = "GeographicalRegion"
region["contains"] = [subregion]

distributed_hierarchy = [region]

print(json.dumps(distributed_hierarchy, indent=4))

[
    {
        "container": "GeographicalRegion",
        "contains": [
            {
                "container": "SubGeographicalRegion",
                "contains": [
                    {
                        "container": "Substation",
                        "contains": [
                            {
                                "container": "VoltageLevel",
                                "contains": [
                                    {
                                        "container": "Bay",
                                        "contains": []
                                    }
                                ]
                            },
                            {
                                "container": "Feeder",
                                "contains": []
                            }
                        ]
                    }
                ]
            }
        ]
    }
]


In [10]:
network = NodeBreakerModel(connection=bg, container=geo_region, distributed=True, distributed_hierarchy=distributed_hierarchy)

In [14]:

for sr_area in network.distributed_areas[cim.SubGeographicalRegion].values():
    print("subregion", sr_area.container.name)
    for sub_area in sr_area.distributed_areas[cim.Substation].values():
        print("substation", sub_area.container.name)
        for vl_area in sub_area.distributed_areas[cim.VoltageLevel].values():
            print("voltage level", vl_area.container.name)
        sub_area.get_all_edges(cim.Substation)
            


subregion small
substation maple10bus_sub1
voltage level SUB1_230.0_B1
voltage level SUB1_34.5_B1
voltage level SUB1_115.0_B1
voltage level SUB1_12.47_B1
substation maple10bus_sub2
voltage level SUB2_34.5_B1
voltage level SUB2_4.16_B1
voltage level SUB2_115.0_B1
voltage level SUB2_230.0_B1


In [12]:
len(network.distributed_areas[cim.SubGeographicalRegion])

1

In [13]:
print(sub_area.get_edges_query(cim.Substation))


        PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX cim:  <http://iec.ch/TC57/CIM100#>
        SELECT DISTINCT ?mRID ?attribute ?value ?edge
        WHERE {          
          ?eq r:type cim:Substation.
        VALUES ?mRID { "_FBE667A8-D26D-4B6B-AF9D-74AE20E96041" 
               } 
        ?eq cim:IdentifiedObject.mRID ?mRID.        
        {?eq (cim:|!cim:) ?val.
         ?eq ?attr ?val.}
        UNION
        {?val (cim:|!cim:) ?eq.
         ?val ?attr ?eq.}
        
        {bind(strafter(str(?attr),"#") as ?attribute)}
        {bind(strafter(str(?val),"#") as ?uri)}
        {bind(if(?uri = "", ?val, ?uri) as ?value)}
          
        OPTIONAL {?val a ?classraw.
                  bind(strafter(str(?classraw),"http://iec.ch/TC57/CIM100#") as ?edge_class)
                  {bind(strafter(str(?val),"#") as ?uri)}
                  OPTIONAL {?val cim:IdentifiedObject.mRID ?edge_id.}
                  bind(exists{?val cim:IdentifiedObject.mRID ?edge_id}